In [7]:
import re
import csv

with open("access.log", "r") as log_file:
    log_lines = log_file.readlines()

pattern = r'(\d+\.\d+\.\d+\.\d+) - - \[(.*?)\] "(.*?)" (\d+) (\d+) "(.*?)" "(.*?)"'
matches = [re.findall(pattern, line) for line in log_lines]

log_data = []
for match in matches:
    if match:
        log_data.append(list(match[0]))

with open('log_data.csv', mode='w', newline='') as csv_file:
    fieldnames = ['Adresse_IP', 'Date_Heure', 'Date', 'Heure', 'Method_HTTP', 'Code_Reponse', 'Code_HTTP', 'URL']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for item in log_data:
        date_heure = re.search(r'(\d+/\w+/\d+:\d+:\d+:\d+ \+\d+)', item[1])
        date_heure_str = date_heure.group(1) if date_heure else ""
        method_http = re.search(r'(GET|POST)', item[2])
        method = method_http.group(1) if method_http else ""
        date, _, heure = date_heure_str.partition(':')
        date = date.strip()
        heure = heure.strip()

        writer.writerow({'Adresse_IP': item[0], 'Date_Heure': item[1], 'Date': date, 'Heure': heure, 'Method_HTTP': method, 'Code_Reponse': item[3], 'Code_HTTP': item[4], 'URL': item[5]})

In [8]:
import pandas as pd
data = pd.read_csv('log_data.csv', encoding='ISO-8859-1')
data = data.drop(['Code_HTTP', 'Date_Heure'], axis=1)
data['Heure'] = data['Heure'].str.extract(r'(\d+:\d+:\d+)')
data.to_csv('log_data.csv', index=False)

In [11]:
data  = pd.read_csv('log_data.csv', encoding='ISO-8859-1')
data


,Adresse_IP,Date,Heure,Method_HTTP,Code_Reponse,URL
0,66.249.64.2,14/Oct/2023,00:00:17,GET,500,https://iteam-s.mg/
1,205.210.31.93,14/Oct/2023,00:06:24,GET,200,-
2,198.235.24.22,14/Oct/2023,00:17:36,GET,301,-
3,198.235.24.22,14/Oct/2023,00:17:37,GET,200,http://144.126.142.237:80/
4,205.210.31.145,14/Oct/2023,00:21:08,GET,301,-
...,...,...,...,...,...,...
1169,203.192.210.158,14/Oct/2023,15:19:47,NaN,400,-
1170,198.235.24.53,14/Oct/2023,15:34:24,GET,302,-
1171,198.235.24.37,14/Oct/2023,15:39:31,GET,301,-
1172,198.235.24.37,14/Oct/2023,15:39:32,GET,404,http://144.126.142.237:80/


In [17]:
from datetime import datetime
data['Date'] = pd.to_datetime(data['Date'])
data['Day'] = data['Date'].apply(lambda x: x.strftime("%A"))
data = data[['Adresse_IP', 'Day', 'Date', 'Heure', 'Method_HTTP', 'Code_Reponse', 'URL']]
data

,Adresse_IP,Date,Heure,Method_HTTP,Code_Reponse,URL,Day
0,66.249.64.2,2023-10-14,00:00:17,GET,500,https://iteam-s.mg/,Saturday
1,205.210.31.93,2023-10-14,00:06:24,GET,200,-,Saturday
2,198.235.24.22,2023-10-14,00:17:36,GET,301,-,Saturday
3,198.235.24.22,2023-10-14,00:17:37,GET,200,http://144.126.142.237:80/,Saturday
4,205.210.31.145,2023-10-14,00:21:08,GET,301,-,Saturday
...,...,...,...,...,...,...,...
1169,203.192.210.158,2023-10-14,15:19:47,NaN,400,-,Saturday
1170,198.235.24.53,2023-10-14,15:34:24,GET,302,-,Saturday
1171,198.235.24.37,2023-10-14,15:39:31,GET,301,-,Saturday
1172,198.235.24.37,2023-10-14,15:39:32,GET,404,http://144.126.142.237:80/,Saturday


In [12]:
data['Code_Reponse'].value_counts()

Code_Reponse
200    485
404    307
301    224
302     64
502     53
400     36
500      2
304      1
405      1
499      1
Name: count, dtype: int64

In [14]:
data['Adresse_IP'].value_counts()

Adresse_IP
94.228.169.182     214
3.137.203.71       162
3.84.125.222        35
3.91.29.0           35
154.126.11.106      35
                  ... 
198.199.101.64       1
136.244.84.156       1
198.235.24.98        1
165.227.174.100      1
205.210.31.72        1
Name: count, Length: 320, dtype: int64